In [1]:
import numpy as np
import helper_func
from sklearn import linear_model
import cPickle as pickle
import models
from sklearn.metrics.pairwise import cosine_similarity as cosdist
import edward as ed
from edward.models import Poisson,Gamma

In [3]:
PIK = "../models/eur_sm_invgaus_po_100.dat"
with open(PIK, "rb") as f:
    u,_,vy= pickle.load(f)
f.close()
dataset = 'multi_eur'
x_train,y_train,x_test,y_test = helper_func.load_data(dataset)
k = u.shape[1]
w = np.zeros(shape=[x_train.shape[1],u.shape[1]])

In [4]:
temp = np.matmul(u,vy)
result = 0.0
for i in range(0,temp.shape[0]):
    result += helper_func.patk(temp[i],y_train[i],1)
result /= temp.shape[0]
print result

0.05375


In [5]:
for i in range(0,k):
    print i
    reg = linear_model.Ridge(alpha = 0.0)
    reg.fit(x_train,u[:,i])
    w[:,i] = reg.coef_
u_hat = np.matmul(x_train,w)
u_test = np.matmul(x_test,w)

0


/home/jarvis/anaconda2/envs/bml/lib/python2.7/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [7]:
nb = 150
result = 0.0
for i in range(0,u_test.shape[0]):
    dist = cosdist(np.reshape(u_test[i],newshape=[1,-1]),u_hat)
    indices = np.argsort(dist)[:,:nb]
    predicted = np.zeros(shape=y_train.shape[1],dtype=np.float64)
    for j in range(0,nb):
        predicted += y_train[indices[0,j]]
    predicted /= nb
    result += helper_func.patk(predicted,y_test[i],1)
result /= u_test.shape[0]
print result

0.0620796689084
